In [15]:
import cirq
import qiskit
from qiskit import QuantumCircuit

### Cirq: Quantum Fourier Transform Circuits

In [8]:
"""Pass in the qubits into the quantum Fourier transform circuit."""
def quantum_fourier_transform(qubits) -> None:
    n = len(qubits)
    circuit = cirq.Circuit()

    for i in range(n):
        circuit.append(cirq.H(qubits[i]))
        for j in range(i + 1, n):
            circuit.append(cirq.CZ(qubits[j], qubits[i]) ** (1 / (2 ** (j - i))))
    
    return circuit


num_qubits = 5
qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
circuit = quantum_fourier_transform(qubits)
print(circuit)

                  ┌───────┐   ┌────────────┐   ┌───────────────┐   ┌────────────┐   ┌───────┐
0: ───H───@────────@───────────@────────────────@───────────────────────────────────────────────────────────
          │        │           │                │
1: ───────@^0.5────┼─────H─────┼──────@─────────┼───────@───────────@───────────────────────────────────────
                   │           │      │         │       │           │
2: ────────────────@^0.25──────┼──────@^0.5─────┼───────┼─────H─────┼──────@─────────@──────────────────────
                               │                │       │           │      │         │
3: ────────────────────────────@^(1/8)──────────┼───────@^0.25──────┼──────@^0.5─────┼─────H────@───────────
                                                │                   │                │          │
4: ─────────────────────────────────────────────@^(1/16)────────────@^(1/8)──────────@^0.25─────@^0.5───H───
                  └───────┘   └────────────┘   └────────

### Qiskit: Quantum Fourier Transform Circuits

In [17]:
"""Pass the qubits into the quantum Fourier transform circuit"""
def quantum_fourier_transform(n) -> QuantumCircuit:
    circuit = QuantumCircuit(n)

    for i in range(n):
        circuit.h(i)
        for j in range(i + 1, n):
            circuit.cp(1 / (2 ** (j - i)), j, i)
    
    return circuit

num_qubits = 5
circuit = quantum_fourier_transform(num_qubits)
print(circuit.draw())

     ┌───┐                                                                  »
q_0: ┤ H ├─■────────■──────────────■───────────────────■────────────────────»
     └───┘ │P(0.5)  │        ┌───┐ │                   │                    »
q_1: ──────■────────┼────────┤ H ├─┼──────────■────────┼───────────■────────»
                    │P(0.25) └───┘ │          │P(0.5)  │           │        »
q_2: ───────────────■──────────────┼──────────■────────┼───────────┼────────»
                                   │P(0.125)           │           │P(0.25) »
q_3: ──────────────────────────────■───────────────────┼───────────■────────»
                                                       │P(0.0625)           »
q_4: ──────────────────────────────────────────────────■────────────────────»
                                                                            »
«                                                           
«q_0: ──────────────────────────────────────────────────────
«                   